In [6]:
import numpy as np
import pandas as pd
import scipy.io
from sklearn.metrics import accuracy_score

# a

In [2]:
import math

def find_cluster(X, k, mu):
    for i, x in X.iterrows():
        best_distance = math.inf
        best_cluster = None
        for j in range(k):
            new_distance = np.linalg.norm(x-mu.iloc[j])
            if new_distance < best_distance:
                best_distance = new_distance
                best_cluster = j
            
        X.at[i, "cluster"] = best_cluster
            
    return X

In [3]:
def calculate_means(X, k):
    data_list = []
    for i in range(k):
        data_list.append(X[X["cluster"]==i].mean())
    
    mu = pd.DataFrame(data_list)
    return mu

In [4]:
def sse_error(X, k):
    error = 0
    for i in range(k):
        X_cluster = X[X["cluster"]==i]
        m = X_cluster.mean()
        for index, x in X_cluster.iterrows():
            error += np.linalg.norm(x-m)**2
    return error

In [32]:
import math

def kmeanscluster(X, k, mu, tol, maxIter):
    prev_error = math.inf
    for iter in range(maxIter):
        X = find_cluster(X, k, mu)
        mu = calculate_means(X, k)
        new_error = sse_error(X, k)
        if prev_error - new_error < tol:
            break
        prev_error = new_error
    return X

In [25]:
def accuracy(C, Y):
    C["class"] = Y

    cluster_class = C.groupby("cluster").agg({"class":max}).reset_index()
    
    true_list = []
    predict_list = []
    for item, c in C.iterrows():
        true_class = c["class"]
        predict_class = cluster_class[cluster_class["cluster"]==c["cluster"]]["class"]
        true_list.append(true_class)
        predict_list.append(predict_class)
    
    score = accuracy_score(true_list, predict_list)

    return score

# b

In [29]:
for t in range(5):
    X = pd.read_csv("breast_data.csv", header=None)

    Y = pd.read_csv("breast_labels.csv", header=None)

    k = 2

    mu = X.sample(n=k)

    C = kmeanscluster(X=X, k=k, mu=mu, tol=100, maxIter=50)

    print(accuracy(C, Y))

0 85840970.30001347
0.37258347978910367


# c

In [31]:
X = pd.read_csv("breast_data.csv", header=None)

Y = pd.read_csv("breast_labels.csv", header=None)

k = 2

mu = pd.DataFrame(scipy.io.loadmat('init_mu')["mu_init"].T)

C = kmeanscluster(X=X, k=k, mu=mu, tol=100, maxIter=50)

print(accuracy(C, Y))

0 255880220.3749644
5 77949789.73226753
0.37258347978910367
